In [1]:
import pymongo
print(pymongo.__version__)

3.12.0


#### Connet to mongoDB and import row data

In [3]:
client = "mongodb://[your account]:[password]]@[your IP address]/raw_data_for_project"
conn = pymongo.MongoClient(client, serverSelectionTimeoutMS=5000)

db = conn.raw_data_for_project # Choose your database
print(db.list_collection_names())

collection = db.ptt

['pixnet_hotel_taiwan', 'travelYam_hotel', 'hotels_com', 'test', 'ptt', 'pixnet_hotel', 'backpackers_voyage', 'dcard', 'ifoodie', 'booking_hotel', 'backpackers_hotel']


In [9]:
print(db.pixnet_hotel_taiwan.find().count())

6623


<ipython-input-9-86f3c98fb811>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(db.pixnet_hotel_taiwan.find().count())


In [1]:
import pandas as pd
import json
import numpy
import re
import jieba
import jieba.analyse
import time

### Clean row data from PTT

In [3]:
# Getting row data(json)
raw_ptt = pd.json_normalize([i for i in db.ptt.find({})])
ptt = raw_ptt.copy()
ptt.columns

ptt = ptt.drop("id", axis=1).drop_duplicates(subset=["title"])
ptt = ptt[~ptt["title"].str.contains("請益")] # drop context include keywords: 請益

Index(['_id', 'title', 'content', 'link'], dtype='object')

> Creat jieba dict w/ hotelList and stopWords

In [5]:
# Set jieba dictoinary and stopwords
jieba.load_userdict("./hotelList.txt")
jieba.analyse.set_stop_words('./stopwords.txt')
# Hotel Name List
hotelName = pd.read_csv("./hotelList.txt", sep = "\n")
hotelList = [i for i in hotelName.Name]
# Stop words list
stopWords = pd.read_csv('./stopwords.txt', sep = "\n")
stopWords = [word for word in stopWords.WORD]
stopWords.append('\n')
stopWords.append('\n\n')
stopWords.append(' ')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\TIBAME~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.592 seconds.
Prefix dict has been built successfully.


In [6]:
# from content extract author 
ptt['author'] = ptt['content'].apply(lambda x : x.split(' (')[0])

# check author have 作者, if not drop data
ptt = ptt[ptt['author'].str.contains('作者')]
ptt['author'] = ptt['author'].apply(lambda x : x.split('作者')[1])

# from content extract time
ptt['time'] = ptt['content'].apply(lambda x : x.split('\n')[0])

# check author have 時間, if not drop data
ptt = ptt[ptt['time'].str.contains('時間')]
ptt['time'] =ptt['time'].apply(lambda x : x.split('時間')[1])
ptt['time'] =ptt['time'].apply(lambda x : x if len(x) == 24 else None)
ptt = ptt.dropna(subset=['time'])

# origin time
ptt['time'] = ptt['time'].apply(lambda x : time.strptime(x, "%a %b %d %H:%M:%S %Y")) # 轉成時間元組
ptt['time'] = ptt['time'].apply(lambda x : time.strftime('%Y-%m-%d', x)) 

In [7]:
def clean_word(row):
    words = re.findall('\w+', row)
    words_new = '|'.join(words)
    return words_new.replace('臺', '台')

ptt['title'] = ptt['title'].apply(clean_word)
ptt['article'] = ptt['content'].apply(clean_word)

In [8]:
# try get hotel name from title
def extracthotel(row):
    raw_cut = jieba.lcut(row)
    name = [hotel for hotel in raw_cut if hotel in hotelList]
    if name != []:
        return '|'.join(name)
    else:
        return None
    

ptt['hotel_name'] = ptt['title'].apply(extracthotel)

In [9]:
ptt = ptt.rename(columns={'content': 'article'})

In [10]:
ptt

,title,article,link,author,time,hotel_name
1,心得|宜蘭|蘇澳|煙波大飯店蘇澳四季雙泉館,作者mandywei|喂喂不在家|看板Hotel標題|心得|蘇澳|煙波大飯店蘇澳四季雙泉館時...,https://www.ptt.cc/bbs/Hotel/M.1632597213.A.2D...,mandywei,2021-09-26,煙波大飯店
2,心得|宜蘭|麗翔酒店礁溪館|南洋風戶外溫泉池,作者terry910134|懶人愛吃喝|看板Hotel標題|心得|宜蘭|麗翔酒店礁溪館|南洋...,https://www.ptt.cc/bbs/Hotel/M.1632021142.A.7F...,terry910134,2021-09-19,麗翔酒店
3,心得|宜蘭|礁溪山泉大飯店住宿含早餐與宵夜,作者faigo3ppp|Dr|Lai|看板Hotel標題|心得|宜蘭|礁溪山泉大飯店住宿含早...,https://www.ptt.cc/bbs/Hotel/M.1631501681.A.84...,faigo3ppp,2021-09-13,山泉大飯店
4,心得|宜蘭|悅川酒店,作者shmily929|呆呆|看板Hotel標題|心得|宜蘭|悅川酒店時間Fri|Sep|1...,https://www.ptt.cc/bbs/Hotel/M.1631275274.A.07...,shmily929,2021-09-10,悅川酒店
5,心得|宜蘭|頭城時尚會館|一泊二食|6680元,作者sailly|人間好時節|看板Hotel標題|心得|宜蘭頭城時尚會館|一泊二食|6680...,https://www.ptt.cc/bbs/Hotel/M.1630466309.A.27...,sailly,2021-09-01,頭城時尚會館
...,...,...,...,...,...,...
4395,抱怨|台北瑞格商務會館,作者lucy0627|滷蛋芽芽|看板Hotel標題|抱怨|台北瑞格商務會館時間Thu|Jun...,https://www.ptt.cc/bbs/Hotel/M.1214412821.A.9E...,lucy0627,2008-06-26,None
4396,心得|台北市華美大飯店,作者hengheng|研究程式中|看板Hotel標題|心得|台北市華美大飯店時間Tue|Ju...,https://www.ptt.cc/bbs/Hotel/M.1214237933.A.F8...,hengheng,2008-06-24,None
4397,討論|台北市小而美的旅館,作者kcmytaco|屋頂上的提琴手|看板Hotel標題|討論|台北市小而美的旅館時間Thu...,https://www.ptt.cc/bbs/Hotel/M.1213886890.A.38...,kcmytaco,2008-06-19,None
4398,心得|台北市|百悅商務旅館,作者sanguine0814|看板Hotel標題|心得|台北市|百悅商務旅館時間Wed|Ju...,https://www.ptt.cc/bbs/Hotel/M.1213792466.A.C8...,sanguine0814,2008-06-18,None


In [ ]:
ptt.to_csv('ptt_travel.csv',encoding="utf_8_sig", index=False)

In [ ]:
df = pd.read_csv('./ptt_travel.csv', encoding = 'utf-8')
df

In [ ]:
count = 0
for i in ptt['hotel_name']:
    if i is None:
        count += 1
print(count)

### Clean row data from backpacker

In [11]:
raw_voyage =pd.json_normalize([i for i in db.backpackers_voyage.find({})])
raw_hotel =pd.json_normalize([i for i in db.backpackers_hotel.find({})])# Become a dataframe already
voyage = raw_voyage.copy()
hotel = raw_hotel.copy()
hotel.columns

Index(['_id', 'author', 'author_ID', 'post', 'article_url', 'time', 'article',
       'imgURL', 'reply'],
      dtype='object')

In [12]:
# combine two dataframes(hotel and voyage), use outer join so pull all datas
backpackers = pd.concat([hotel, voyage], join='outer', ignore_index= True) 
backpackers

,_id,author,author_ID,post,article_url,time,article,imgURL,reply
0,616a67686ae778940dcf88cb,安001,1610553,尋求台北短租,https://www.backpackers.com.tw/forum/showthrea...,"2021-10-09, 13:39","[你們好, 因疫情故時常居家上班, 此次穩定要回公司上班, 但不確定公司是否還會不會再居家,...",無,"[引用, 作者, 原文章, 你們好, 因疫情故時常居家上班, 此次穩定要回公司上班, 但不確..."
1,616a67686ae778940dcf88cc,hanayome000,1608032,[反推]在booking上名不符實，又人間蒸發的業主,https://www.backpackers.com.tw/forum/showthrea...,"2021-08-16, 21:43","[更新, 詐騙, 反推, 假民宿, 假地址無誤, 詳見下列圖片補充說明, 我已經投訴到消保官...",[https://sa.bbkz.net/forum/attachment.php?s=4d...,"[去年因為無法出國在台灣玩, 也真遇過幾間要你先出錢的, 奇怪了, 我沒來你就從信用卡扣掉不..."
2,616a67686ae778940dcf88cd,MinakoTseng,517889,JR東日本大飯店台北開箱,https://www.backpackers.com.tw/forum/showthrea...,"2021-08-24, 14:28","[如果說去年夏天最令人興奮的事情是, 開幕的話, 那今年夏天最令人期待的應該就是, 東日本大...",[https://sa1.bbkz.net/forum/attachment.php?s=4...,無
3,616a67686ae778940dcf88ce,christin,1607310,租屋,https://www.backpackers.com.tw/forum/showthrea...,"2021-08-02, 19:06","[月底要至台北榮總上班, 租屋預算, 租屋地點希望走路到醫院10分鐘內, 麻煩有合適的推薦一...",無,無
4,616a67686ae778940dcf88cf,christin,1607310,租屋,https://www.backpackers.com.tw/forum/showthrea...,"2021-08-02, 19:01","[月底要到北榮上班想租屋預算約, 希望租屋處走路至醫院10分鐘謝謝]",無,無
...,...,...,...,...,...,...,...,...,...
5260,616a53aa678242ff812022d9,jackls1533,25337,951209 宜蘭二日遊遊記,https://www.backpackers.com.tw/forum/showthrea...,"2007-01-18, 13:05","[去年12月初去了趟宜蘭玩了兩天, 當時規劃最主要的景點是福山植物園, 一個月前就登記, 終...",無,無
5261,616a53aa678242ff812022da,wsykinki,26990,2006福山植物園一日遊,https://www.backpackers.com.tw/forum/showthrea...,"2006-09-08, 13:01","[我寫了有關福山植物園的遊記, 害羞, 大家有興趣, 可以上去瞧瞧]",無,"[花了快一小時, 終於看完了, 寫得真的不錯喔, 在這邊也給你鼓勵, 掌聲, 分喔, 福山植..."
5262,616a53aa678242ff812022db,nmnl,43973,"宜蘭一日遊最佳選擇""不老部落""",https://www.backpackers.com.tw/forum/showthrea...,"2006-07-09, 18:28","[前一陣子在公視的節目上看到, 不老部落, 的介紹, 描述著一個有關泰雅女婿放棄台北的生活,...",無,無
5263,616a53aa678242ff812022dc,jackls1533,25337,950506 北橫、宜蘭遊,https://www.backpackers.com.tw/forum/showthrea...,"2006-05-12, 22:18","[前言, 老婆因工作之故, 星期六要上半天班, 星期一補休, 難得有非假日能出遊, 當然要把...",無,"[行程, 第一天, 淡水, 關渡橋, 北二高中和交流道, 大溪交流道, 角板山, 羅浮橋, ..."


In [13]:
# 先將沒有用到的column drop
# backpackers = backpackers.drop(['_id','author_ID'], axis = 1).drop_duplicates(subset=['post'])

# first transfom to string because Error with 'float'
backpackers['post']=backpackers['post'].astype('string')
# Drop 'post' column with some keywords cuz not what we need 
backpackers = backpackers[~backpackers['post'].str.contains('求|租|一日|短期|高雄|基隆')]
# Clean sentence only keep A-Z, number and chinese
backpackers['post'] = backpackers['post'].apply(clean_word)

# Tiem formate to YYYY-MM-DD
backpackers['time'] = backpackers['time'].apply(lambda x : time.strptime(x, "%Y-%m-%d, %H:%M")) # 轉成時間元組
backpackers['time'] = backpackers['time'].apply(lambda x : time.strftime('%Y-%m-%d', x)) 

<ipython-input-13-6b5d0515da92>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backpackers['post'] = backpackers['post'].apply(clean_word)
<ipython-input-13-6b5d0515da92>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backpackers['article'] = backpackers['article'].apply(lambda x : '|'.join(x))


In [14]:
def extracthotel(row):
    raw_cut = jieba.lcut(row)
    name = [hotel for hotel in raw_cut if hotel in hotelList]
    if name != []:
        return '|'.join(name)
    else:
        return None

backpackers['hotel_name'] = backpackers['article'].apply(extracthotel)

In [15]:
# Rename, same as ptt DataFrame title colums name
backpackers = backpackers.rename(columns={'post': 'title'})

In [ ]:
count = 0
for i, x in enumerate(backpackers['hotel_name']):
    if x is not None:
        count += 1
        print(i)
        print(x)
        print() 
print(count)

In [ ]:
backpackers.loc[5258, 'article']
# backpackers

In [ ]:
count = 0
for i in backpackers['hotel_name']:
    if i is None:
        count += 1
print(count)

#### Final combin ptt and backpackers DataFrame 

In [16]:
hotel_articles = pd.concat([ptt, backpackers], join='outer', ignore_index= True)
hotel_articles

,title,article,link,author,time,hotel_name,_id,author_ID,article_url,imgURL,reply
0,心得|宜蘭|蘇澳|煙波大飯店蘇澳四季雙泉館,作者mandywei|喂喂不在家|看板Hotel標題|心得|蘇澳|煙波大飯店蘇澳四季雙泉館時...,https://www.ptt.cc/bbs/Hotel/M.1632597213.A.2D...,mandywei,2021-09-26,煙波大飯店,NaN,NaN,NaN,NaN,NaN
1,心得|宜蘭|麗翔酒店礁溪館|南洋風戶外溫泉池,作者terry910134|懶人愛吃喝|看板Hotel標題|心得|宜蘭|麗翔酒店礁溪館|南洋...,https://www.ptt.cc/bbs/Hotel/M.1632021142.A.7F...,terry910134,2021-09-19,麗翔酒店,NaN,NaN,NaN,NaN,NaN
2,心得|宜蘭|礁溪山泉大飯店住宿含早餐與宵夜,作者faigo3ppp|Dr|Lai|看板Hotel標題|心得|宜蘭|礁溪山泉大飯店住宿含早...,https://www.ptt.cc/bbs/Hotel/M.1631501681.A.84...,faigo3ppp,2021-09-13,山泉大飯店,NaN,NaN,NaN,NaN,NaN
3,心得|宜蘭|悅川酒店,作者shmily929|呆呆|看板Hotel標題|心得|宜蘭|悅川酒店時間Fri|Sep|1...,https://www.ptt.cc/bbs/Hotel/M.1631275274.A.07...,shmily929,2021-09-10,悅川酒店,NaN,NaN,NaN,NaN,NaN
4,心得|宜蘭|頭城時尚會館|一泊二食|6680元,作者sailly|人間好時節|看板Hotel標題|心得|宜蘭頭城時尚會館|一泊二食|6680...,https://www.ptt.cc/bbs/Hotel/M.1630466309.A.27...,sailly,2021-09-01,頭城時尚會館,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4611,我的宜蘭兩日遊記,以下是我和我朋友到宜蘭玩的一些景點|其中有住宿的也有吃喝玩樂的|我不敢說我的行程是頂好的|但...,NaN,presotea,2008-06-07,中冠礁溪大飯店,616a53aa678242ff812022cb,182547,https://www.backpackers.com.tw/forum/showthrea...,[https://sa.bbkz.net/forum/attachment.php?s=6b...,"[有沒有去買宜蘭餅, 創始店那家, 還有好吃三星蔥大餐, 宜蘭餅的創始店, 是有去吃, 但是..."
4612,宜蘭新寮瀑布,雪山隧道通車後|大家都好愛往宜蘭跑|去玩一|二天已經是很基本的遊程|還有朋友特地驅車去礁溪吃...,NaN,clitier,2007-05-17,None,616a53aa678242ff812022d2,74748,https://www.backpackers.com.tw/forum/showthrea...,無,"[引用, 作者, 雪山隧道通車後, 大家都好愛往宜蘭跑, 去玩一, 二天已經是很基本的遊程,..."
4613,翠峰湖|太平山,天一夜的礁溪羅東|太平山之旅|第一天玩完礁溪|羅東後晚上從羅東開7丙轉泰雅大橋上玉蘭茶園的櫻...,NaN,skyc,2007-06-03,太平山莊|太平山莊,616a53aa678242ff812022d4,94700,https://www.backpackers.com.tw/forum/showthrea...,無,"[光看文字感覺就很讚, 彷彿就有身歷其境的感覺, 小弟我雖然已經去宜蘭蠻多次了, 但每次都無..."
4614,初訪宜蘭,通過了雪山隧道|壯闊的蘭陽平原呈現在眼前|第一次到訪宜蘭|我們也不知道該從哪兒開始|所以就順...,NaN,ㄚ玲,2007-04-09,None,616a53aa678242ff812022d7,63491,https://www.backpackers.com.tw/forum/showthrea...,無,"[上次我也有到傳藝中心去, 還不賴啦, 尤其是它的糖胡蘆, 好好吃喔, 念念不忘呀]"


In [17]:
hotel_articles.to_csv('hotel_articles.csv',encoding="utf-8", index=False)
print("====儲存成功====")

====儲存成功====


In [18]:
df = pd.read_csv('hotel_articles.csv',encoding='utf-8')
df

,title,article,link,author,time,hotel_name,_id,author_ID,article_url,imgURL,reply
0,心得|宜蘭|蘇澳|煙波大飯店蘇澳四季雙泉館,作者mandywei|喂喂不在家|看板Hotel標題|心得|蘇澳|煙波大飯店蘇澳四季雙泉館時...,https://www.ptt.cc/bbs/Hotel/M.1632597213.A.2D...,mandywei,2021-09-26,煙波大飯店,NaN,NaN,NaN,NaN,NaN
1,心得|宜蘭|麗翔酒店礁溪館|南洋風戶外溫泉池,作者terry910134|懶人愛吃喝|看板Hotel標題|心得|宜蘭|麗翔酒店礁溪館|南洋...,https://www.ptt.cc/bbs/Hotel/M.1632021142.A.7F...,terry910134,2021-09-19,麗翔酒店,NaN,NaN,NaN,NaN,NaN
2,心得|宜蘭|礁溪山泉大飯店住宿含早餐與宵夜,作者faigo3ppp|Dr|Lai|看板Hotel標題|心得|宜蘭|礁溪山泉大飯店住宿含早...,https://www.ptt.cc/bbs/Hotel/M.1631501681.A.84...,faigo3ppp,2021-09-13,山泉大飯店,NaN,NaN,NaN,NaN,NaN
3,心得|宜蘭|悅川酒店,作者shmily929|呆呆|看板Hotel標題|心得|宜蘭|悅川酒店時間Fri|Sep|1...,https://www.ptt.cc/bbs/Hotel/M.1631275274.A.07...,shmily929,2021-09-10,悅川酒店,NaN,NaN,NaN,NaN,NaN
4,心得|宜蘭|頭城時尚會館|一泊二食|6680元,作者sailly|人間好時節|看板Hotel標題|心得|宜蘭頭城時尚會館|一泊二食|6680...,https://www.ptt.cc/bbs/Hotel/M.1630466309.A.27...,sailly,2021-09-01,頭城時尚會館,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4611,我的宜蘭兩日遊記,以下是我和我朋友到宜蘭玩的一些景點|其中有住宿的也有吃喝玩樂的|我不敢說我的行程是頂好的|但...,NaN,presotea,2008-06-07,中冠礁溪大飯店,616a53aa678242ff812022cb,182547.0,https://www.backpackers.com.tw/forum/showthrea...,['https://sa.bbkz.net/forum/attachment.php?s=6...,"['有沒有去買宜蘭餅', '創始店那家', '還有好吃三星蔥大餐', '宜蘭餅的創始店', ..."
4612,宜蘭新寮瀑布,雪山隧道通車後|大家都好愛往宜蘭跑|去玩一|二天已經是很基本的遊程|還有朋友特地驅車去礁溪吃...,NaN,clitier,2007-05-17,NaN,616a53aa678242ff812022d2,74748.0,https://www.backpackers.com.tw/forum/showthrea...,無,"['引用', '作者', '雪山隧道通車後', '大家都好愛往宜蘭跑', '去玩一', '二..."
4613,翠峰湖|太平山,天一夜的礁溪羅東|太平山之旅|第一天玩完礁溪|羅東後晚上從羅東開7丙轉泰雅大橋上玉蘭茶園的櫻...,NaN,skyc,2007-06-03,太平山莊|太平山莊,616a53aa678242ff812022d4,94700.0,https://www.backpackers.com.tw/forum/showthrea...,無,"['光看文字感覺就很讚', '彷彿就有身歷其境的感覺', '小弟我雖然已經去宜蘭蠻多次了',..."
4614,初訪宜蘭,通過了雪山隧道|壯闊的蘭陽平原呈現在眼前|第一次到訪宜蘭|我們也不知道該從哪兒開始|所以就順...,NaN,ㄚ玲,2007-04-09,NaN,616a53aa678242ff812022d7,63491.0,https://www.backpackers.com.tw/forum/showthrea...,無,"['上次我也有到傳藝中心去', '還不賴啦', '尤其是它的糖胡蘆', '好好吃喔', '念..."


In [ ]:
ptt = {}
for data in ptt:

    hotel_name = jieba.analyse.extract_tags(data['title'])
    article = re.findall('[\u4e00-\u9fa5]+\d*[\u4e00-\u9fa5]+', data['content'])
#     article_1 = '&'.join(article)
    if len(article) < 20:
        article = 0
#     article = [t for t in jieba.cut(data['content']) if t not in stopWords]
#     print(article)

#     article = data['content'].split(' (')[1]
    author = data['content'].split(' (')[0]
    if len(author) > 100: # 有時候split出來會是文章，表示格式不正確
        author = 0
    time = data['content'].split('\n')[0].split('時間')[-1]
    if len(time) != 24:
        time = 0
    
    ptt['title'] = hotel_name
    ptt['author'] = author
    ptt['time'] = time
    ptt['article'] = article
    
    ptt